In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
uid_train = pd.read_csv('../data/uid_train.txt',sep='\t',header=None,names=('uid','label'))
voice_train = pd.read_csv('../data/voice_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})
sms_train = pd.read_csv('../data/sms_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})
wa_train = pd.read_csv('../data/wa_train.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

/home/suhuiling/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
voice_test = pd.read_csv('../data/voice_test_b.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})
sms_test = pd.read_csv('../data/sms_test_b.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})
wa_test = pd.read_csv('../data/wa_test_b.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

In [4]:
prefix = np.array(['u'])
uid_num = np.arange(7000,10000)
uid_num_char = uid_num.astype('U')
uid_num_str = np.core.defchararray.add(prefix, uid_num_char)
uid_test = pd.DataFrame(uid_num_str, columns=['uid'])
uid_test.to_csv('../data/uid_test_a.txt',index=None)

In [5]:
voice = pd.concat([voice_train,voice_test],axis=0)
sms = pd.concat([sms_train,sms_test],axis=0)
wa = pd.concat([wa_train,wa_test],axis=0)

In [6]:
voice.start_time = voice.start_time.astype(int)
voice.end_time = voice.end_time.astype(int)
voice['date'] = voice.start_time//1000000
voice['hour'] = voice.start_time%1000000//10000

sms.start_time = sms.start_time.astype(int)
sms['date'] = sms.start_time//1000000
sms['hour'] = sms.start_time%1000000//10000

wa.date = wa.date.fillna(0).astype(int)
wa.up_flow = wa.up_flow.fillna(0).astype(int)
wa.down_flow = wa.down_flow.fillna(0).astype(int)
wa.visit_dura = wa.visit_dura.fillna(0).astype(int)
wa.visit_cnt = wa.visit_cnt.fillna(0).astype(int)

### 通话记录

In [7]:
voice['voice_dura']=abs(voice.end_time.astype('int')-voice.start_time.astype('int'))
# 不同的电话号码数/电话总数
voice_opp_num = voice.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('voice_opp_num_').reset_index()

# 不同的电话号码头三位的数量
voice_opp_head=voice.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).add_prefix('voice_opp_head_').reset_index()

# 每种电话长度的通话次数
voice_opp_len=voice.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('voice_opp_len_').reset_index().fillna(0)

# 每种类型通话的次数
voice_call_type = voice.groupby(['uid','call_type'])['uid'].count().unstack().add_prefix('voice_call_type_').reset_index().fillna(0)

# 每种类型通话的平均时长
voice_dura_type = voice.groupby(['uid','call_type'])['voice_dura'].mean().unstack().add_prefix('voice_dura_type_').reset_index().fillna(0)

# 接入/打出的电话总数
voice_in_out = voice.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('voice_in_out_').reset_index().fillna(0)

# 通话时长的各统计量
voice_dura = voice.groupby(['uid'])['voice_dura'].agg(['std','max','min','median','mean','sum']).add_prefix('voice_dura_').reset_index().fillna(0)

## 每个用户收/发电话的号码的不同号码数
voice_opp_len_inout = voice.groupby(['uid','in_out'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).unstack().add_prefix('voice_opp_inout_num_').reset_index().fillna(0)

# 不同的日期数
voice_day_count = voice.groupby(['uid'])['start_time'].agg({'voice_day_count': lambda x: len(pd.unique(x//1000000))}).reset_index().fillna(0)

# 每天电话量
voice_everyday_count = voice.groupby(['uid','date'])['uid'].count().unstack().add_prefix('voice_everyday_count').reset_index().fillna(0)

# 每个小时段的平均电话量
voice_hour_count = voice.groupby(['uid','hour'])['uid'].count().unstack().add_prefix('voice_hour_count').reset_index().fillna(0)

/home/suhuiling/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until
/home/suhuiling/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
/home/suhuiling/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
/home/suhuiling/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version


## 短信记录

In [8]:
# 不同的短信号码数/电话总数
sms_opp_num = sms.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('sms_opp_num_').reset_index()

# 号码不同头三位的数量
sms_opp_head=sms.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).add_prefix('sms_opp_head_').reset_index()

# 每种号码长度的通话次数
sms_opp_len=sms.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('sms_opp_len_').reset_index().fillna(0)

# 接受/发出短信总数
sms_in_out = sms.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('sms_in_out_').reset_index().fillna(0)

# 不同的日期数
sms_day_count = sms.groupby(['uid'])['start_time'].agg({'sms_day_count': lambda x: len(pd.unique(x//1000000))}).reset_index().fillna(0)

# 每天短信量
sms_everyday_count = sms.groupby(['uid','date'])['uid'].count().unstack().add_prefix('sms_everyday_count').reset_index().fillna(0)

# 每个小时段的平均短信量
sms_hour_count = sms.groupby(['uid','hour'])['uid'].count().unstack().add_prefix('sms_hour_count').reset_index().fillna(0)

/home/suhuiling/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
/home/suhuiling/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """
/home/suhuiling/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


### 网站/APP记录

In [34]:
# 不同wa数量/wa总数
wa_name_cnt = wa.groupby(['uid'])['wa_name'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('wa_name_').reset_index()

# 访问wa次数的各统计量
visit_cnt = wa.groupby(['uid'])['visit_cnt'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_cnt_').reset_index()

# 访问wa时长的各统计量
visit_dura = wa.groupby(['uid'])['visit_dura'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_dura_').reset_index()

# 上传流量的各统计量
up_flow = wa.groupby(['uid'])['up_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_up_flow_').reset_index()

# 下载流量的各统计量
down_flow = wa.groupby(['uid'])['down_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_down_flow_').reset_index()

## 每个用户上/下的流量
w_a_upflow = wa.groupby(['uid','wa_type'])['up_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('w_a_upflow_').unstack().reset_index().fillna(0)

# 不同的日期数
wa_day_count = wa.groupby(['uid'])['date'].agg({'wa_day_count': lambda x: len(pd.unique(x))}).reset_index().fillna(0)

# 每天web上传流量
wa_everyday_web_up_flow = wa[wa.wa_type==0].groupby(['uid','date'])['up_flow'].sum().unstack().add_prefix('wa_everyday_web_up_flow').reset_index().fillna(0)

# 每天app上传流量
wa_everyday_app_up_flow = wa[wa.wa_type==1].groupby(['uid','date'])['up_flow'].sum().unstack().add_prefix('wa_everyday_app_up_flow').reset_index().fillna(0)

# 每天web下载流量
wa_everyday_web_down_flow = wa[wa.wa_type==0].groupby(['uid','date'])['down_flow'].sum().unstack().add_prefix('wa_everyday_web_down_flow').reset_index().fillna(0)

# 每天app下载流量
wa_everyday_app_down_flow = wa[wa.wa_type==1].groupby(['uid','date'])['down_flow'].sum().unstack().add_prefix('wa_everyday_app_down_flow').reset_index().fillna(0)

# 每天web访问时长
wa_everyday_web_visit_dura = wa[wa.wa_type==0].groupby(['uid','date'])['visit_dura'].sum().unstack().add_prefix('wa_everyday_web_visit_dura').reset_index().fillna(0)

# 每天app访问时长
wa_everyday_app_visit_dura = wa[wa.wa_type==1].groupby(['uid','date'])['visit_dura'].sum().unstack().add_prefix('wa_everyday_app_visit_dura').reset_index().fillna(0)

/home/suhuiling/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
/home/suhuiling/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version


In [20]:
# 上传流量最多的w名字
wa_w = wa[wa.wa_type==0]
wa_name = wa_w.groupby(['uid','wa_name'])['up_flow'].sum().unstack().add_prefix('up').reset_index().fillna(0)
wa_uid = wa_name.uid;
wa_name = wa_name.drop(['uid'],axis=1)
col_name = np.arange(0,wa_name.shape[1])
col_name = col_name.astype('U')
wa_name.columns =col_name
wa_name_t = wa_name.T
col_name2 = np.arange(0,wa_name_t.shape[1])
col_name2 = col_name2.astype('U')
wa_name_t.columns =col_name2
ss = wa_name_t.idxmax()
wa_most_up_webname = pd.DataFrame(wa_uid, columns=['uid'])
l = list(ss)
wa_most_up_webname['wa_most_up_webname'] = l

In [21]:
# 上传流量最多的a名字
wa_w = wa[wa.wa_type==1]
wa_name = wa_w.groupby(['uid','wa_name'])['up_flow'].sum().unstack().add_prefix('up').reset_index().fillna(0)
wa_uid = wa_name.uid;
wa_name = wa_name.drop(['uid'],axis=1)
col_name = np.arange(0,wa_name.shape[1])
col_name = col_name.astype('U')
wa_name.columns =col_name
wa_name_t = wa_name.T
col_name2 = np.arange(0,wa_name_t.shape[1])
col_name2 = col_name2.astype('U')
wa_name_t.columns =col_name2
ss = wa_name_t.idxmax()
wa_most_up_appname = pd.DataFrame(wa_uid, columns=['uid'])
l = list(ss)
wa_most_up_appname['wa_most_up_appname'] = l

In [22]:
# 下载流量最多的w名字
wa_w = wa[wa.wa_type==0]
wa_name = wa_w.groupby(['uid','wa_name'])['down_flow'].sum().unstack().add_prefix('down').reset_index().fillna(0)
wa_uid = wa_name.uid;
wa_name = wa_name.drop(['uid'],axis=1)
col_name = np.arange(0,wa_name.shape[1])
col_name = col_name.astype('U')
wa_name.columns =col_name
wa_name_t = wa_name.T
col_name2 = np.arange(0,wa_name_t.shape[1])
col_name2 = col_name2.astype('U')
wa_name_t.columns =col_name2
ss = wa_name_t.idxmax()
wa_most_down_webname = pd.DataFrame(wa_uid, columns=['uid'])
l = list(ss)
wa_most_down_webname['wa_most_down_webname'] = l

In [23]:
# 下载流量最多的a名字
wa_w = wa[wa.wa_type==1]
wa_name = wa_w.groupby(['uid','wa_name'])['down_flow'].sum().unstack().add_prefix('down').reset_index().fillna(0)
wa_uid = wa_name.uid;
wa_name = wa_name.drop(['uid'],axis=1)
col_name = np.arange(0,wa_name.shape[1])
col_name = col_name.astype('U')
wa_name.columns =col_name
wa_name_t = wa_name.T
col_name2 = np.arange(0,wa_name_t.shape[1])
col_name2 = col_name2.astype('U')
wa_name_t.columns =col_name2
ss = wa_name_t.idxmax()
wa_most_down_appname = pd.DataFrame(wa_uid, columns=['uid'])
l = list(ss)
wa_most_down_appname['wa_most_down_appname'] = l

In [24]:
# 访问次数最多的w名字
wa_w = wa[wa.wa_type==0]
wa_name = wa_w.groupby(['uid','wa_name'])['visit_cnt'].sum().unstack().add_prefix('cnt').reset_index().fillna(0)
wa_uid = wa_name.uid;
wa_name = wa_name.drop(['uid'],axis=1)
col_name = np.arange(0,wa_name.shape[1])
col_name = col_name.astype('U')
wa_name.columns =col_name
wa_name_t = wa_name.T
col_name2 = np.arange(0,wa_name_t.shape[1])
col_name2 = col_name2.astype('U')
wa_name_t.columns =col_name2
ss = wa_name_t.idxmax()
wa_most_cnt_webname = pd.DataFrame(wa_uid, columns=['uid'])
l = list(ss)
wa_most_cnt_webname['wa_most_cnt_webname'] = l

In [25]:
# 访问次数最多的a名字
wa_w = wa[wa.wa_type==1]
wa_name = wa_w.groupby(['uid','wa_name'])['visit_cnt'].sum().unstack().add_prefix('cnt').reset_index().fillna(0)
wa_uid = wa_name.uid;
wa_name = wa_name.drop(['uid'],axis=1)
col_name = np.arange(0,wa_name.shape[1])
col_name = col_name.astype('U')
wa_name.columns =col_name
wa_name_t = wa_name.T
col_name2 = np.arange(0,wa_name_t.shape[1])
col_name2 = col_name2.astype('U')
wa_name_t.columns =col_name2
ss = wa_name_t.idxmax()
wa_most_cnt_appname = pd.DataFrame(wa_uid, columns=['uid'])
l = list(ss)
wa_most_cnt_appname['wa_most_cnt_appname'] = l

In [26]:
# 访问时长最多的w名字
wa_w = wa[wa.wa_type==0]
wa_name = wa_w.groupby(['uid','wa_name'])['visit_dura'].sum().unstack().add_prefix('cnt').reset_index().fillna(0)
wa_uid = wa_name.uid;
wa_name = wa_name.drop(['uid'],axis=1)
col_name = np.arange(0,wa_name.shape[1])
col_name = col_name.astype('U')
wa_name.columns =col_name
wa_name_t = wa_name.T
col_name2 = np.arange(0,wa_name_t.shape[1])
col_name2 = col_name2.astype('U')
wa_name_t.columns =col_name2
ss = wa_name_t.idxmax()
wa_most_dura_webname = pd.DataFrame(wa_uid, columns=['uid'])
l = list(ss)
wa_most_dura_webname['wa_most_dura_webname'] = l

In [27]:
# 访问时长最多的a名字
wa_w = wa[wa.wa_type==1]
wa_name = wa_w.groupby(['uid','wa_name'])['visit_dura'].sum().unstack().add_prefix('cnt').reset_index().fillna(0)
wa_uid = wa_name.uid;
wa_name = wa_name.drop(['uid'],axis=1)
col_name = np.arange(0,wa_name.shape[1])
col_name = col_name.astype('U')
wa_name.columns =col_name
wa_name_t = wa_name.T
col_name2 = np.arange(0,wa_name_t.shape[1])
col_name2 = col_name2.astype('U')
wa_name_t.columns =col_name2
ss = wa_name_t.idxmax()
wa_most_dura_appname = pd.DataFrame(wa_uid, columns=['uid'])
l = list(ss)
wa_most_dura_appname['wa_most_dura_appname'] = l

In [35]:
# 将各个特征拼在一起
feature = [voice_opp_num,voice_opp_head,voice_opp_len,voice_call_type,voice_in_out,voice_dura,
           voice_opp_len_inout,voice_dura_type,voice_day_count,voice_everyday_count,voice_hour_count,
           sms_opp_num,sms_opp_head,sms_opp_len,sms_in_out,sms_day_count,sms_everyday_count,sms_hour_count,
           wa_name_cnt,visit_cnt,visit_dura,up_flow,down_flow,w_a_upflow,wa_day_count,wa_everyday_web_up_flow,
           wa_everyday_app_up_flow,wa_everyday_web_down_flow,wa_everyday_app_down_flow,
           wa_everyday_web_visit_dura,wa_everyday_app_visit_dura,wa_most_up_webname,wa_most_down_webname,
           wa_most_cnt_webname,wa_most_dura_webname,wa_most_up_appname,wa_most_down_appname,wa_most_cnt_appname,
           wa_most_dura_appname]

In [36]:
# 将train特征和test特征分离
train_feature = uid_train
for feat in feature:
    train_feature=pd.merge(train_feature,feat,how='left',on='uid')

/home/suhuiling/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:551: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/home/suhuiling/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:862: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  self.right = self.right.drop(right_drop, axis=1)


In [37]:
test_feature = uid_test
for feat in feature:
    test_feature=pd.merge(test_feature,feat,how='left',on='uid')

/home/suhuiling/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:551: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/home/suhuiling/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:862: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  self.right = self.right.drop(right_drop, axis=1)


In [38]:
# 填补Nan
train_feature.fillna(0,inplace=True)
test_feature.fillna(0,inplace=True)

In [39]:
# 保存特征值
train_feature.to_csv('../data/train_featureV1.csv',index=None)
test_feature.to_csv('../data/test_featureV1.csv',index=None)

In [40]:
train_feature.head()

,uid,label,voice_opp_num_unique_count,voice_opp_num_count,voice_opp_head_unique_count,voice_opp_len_3,voice_opp_len_5,voice_opp_len_6,voice_opp_len_7,voice_opp_len_8,...,wa_everyday_app_visit_dura44,wa_everyday_app_visit_dura45,wa_most_up_webname,wa_most_down_webname,wa_most_cnt_webname,wa_most_dura_webname,wa_most_up_appname,wa_most_down_appname,wa_most_cnt_appname,wa_most_dura_appname
0,u0001,0,22.0,79.0,17.0,0.0,3.0,0.0,0.0,0.0,...,1178226.0,831389.0,2814,2813,1097,2814,752,752,752,752
1,u0002,0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2813,9419,9419,9419,0,0,0,0
2,u0003,0,15.0,21.0,10.0,0.0,1.0,0.0,0.0,2.0,...,443061.0,2958897.0,12468,2813,1097,12468,774,774,752,752
3,u0004,0,77.0,254.0,31.0,0.0,1.0,0.0,0.0,12.0,...,935881.0,247476.0,670,2813,12468,12468,752,752,752,752
4,u0005,0,55.0,401.0,28.0,0.0,4.0,0.0,0.0,0.0,...,181149.0,5494.0,7509,3317,7509,7509,752,752,752,752
